# 📘 Week 4 Day 5: 栈、递归与计算图 (Stack & Autograd)

**今日主题**: **逻辑与计算图 (Stack, Recursion & Computational Graphs)**。

  * **算法侧**: **栈 (Stack) 与 递归 (Recursion)**。
  * **AI/PyTorch 侧**: **自动微分 (Autograd)** 的本质。
      * 为什么 `backward()` 叫“反向”传播？因为计算图本质上是一个有向无环图 (DAG)，求导的过程就是利用**栈**的结构，从 Loss 节点倒着往回推导。
      * 深度学习模型（如 PyTorch 的 `nn.Module`）本身就是一种嵌套的**树**形结构。

## 1 核心概念：洋葱是怎么剥开的？

**底层联系 (The Connection):**

  * **栈 (Stack)**: 后进先出 (LIFO)。
  * **前向传播 (Forward)**: $x \to \text{Layer1} \to \text{Layer2} \to Loss$ (像在剥洋葱，一层层算进去)。
  * **反向传播 (Backward)**: $Loss \to \text{Layer2} \to \text{Layer1} \to x$ (像函数递归调用的返回过程，一层层弹栈算梯度)。

## 2 AI 工程实战 (The "Job" Skills)

我们来看一下 PyTorch 是如何记录你的“运算历史”的。

### 2.1 显微镜下的 Autograd (查看 grad\_fn)

请运行这段代码，观察 `grad_fn` 属性，它就是计算图的“链条”。


In [4]:
import torch

# 1. 构建一个简单的计算图
# x -> a -> y -> z
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)

# 乘法节点
a = x * w
# 加法节点
y = a + 10
# 激活函数节点（比如ReLU的简化版：平方）
z = y * y

print(f"z 的值: {z.item()}")

# 2. 追踪grad_fn（链式法则的线索）
print("\n--- 计算图的回溯(Backtracking) ---")
print(f"z.grad_fn: {z.grad_fn}")           # MulBackward (乘法反向)
print(f"y.grad_fn: {y.grad_fn}")           # AddBackward (加法反向)
print(f"a.grad_fn: {a.grad_fn}")           # MulBackward (乘法反向)
print(f"x.grad_fn: {x.grad_fn}")           # None （这是叶子节点，也是我们要求的源头）

# 3. 实际上 backward() 就是在递归调用这些 fn
z.backward()
print(f"\nx 的梯度 (dz/dx): {x.grad}")
# 手推: z = (xw + 10)^2 
# dz/dx = 2(xw+10) * w = 2(2+10)*2 = 48

z 的值: 144.0

--- 计算图的回溯(Backtracking) ---
z.grad_fn: <MulBackward0 object at 0x7452505a3820>
y.grad_fn: <AddBackward0 object at 0x7452505a3790>
a.grad_fn: <MulBackward0 object at 0x7452505a3820>
x.grad_fn: None

x 的梯度 (dz/dx): 48.0


> **💡 思考**: 如果中间断了一环（比如用了 `.detach()`），`grad_fn` 就会变成 `None`，递归回溯就会在那里终止。

## 3 LeetCode 算法突击 (The "Interview" Skills)

今天的三道题，分别对应：**栈的匹配逻辑**、**递归的思维**、**树的遍历**。

### 3.1 有效的括号 (Valid Parentheses) - Easy (经典栈)

  * **LeetCode 链接**: [20. Valid Parentheses](../../LeetCode%20practice/1-50.ipynb)
  * **AI 关联**: 类似于编译器检查代码语法，或者 NLP 中处理嵌套结构。
  * **核心思路**: **栈 (Stack)**。遇到左括号进栈，遇到右括号查栈顶是否匹配。

In [ ]:
class Solution:
    def isValid(self, s: str) -> bool:
        # 映射表，右括号找左括号
        dic = {')': '(', ']': '[', '}': '{'}
        stack = []
        
        for char in s:
            if char in dic:
                # 遇到右括号: 检查栈顶
                # 如果栈空，或者栈顶不匹配，则无效
                top_element = stack.pop() if stack else '#'
                if dic[char] != top_element:
                    return False
            else:
                # 遇到左括号: 进栈
                stack.append(char)
        
        # 最后栈必须为空才算完全闭合
        return not stack

# --- 交互式测试 ---
solution = Solution()
print("=== 测试有效的括号 ===")
print("示例输入: ()[]{} 或 ([)]")
s_input = input("请输入括号字符串: ").strip()

result = solution.isValid(s_input)
print(f"输入: {s_input}")
print(f"是否有效: {result}")

### 3.2 爬楼梯 (Climbing Stairs) - Easy (递归/DP)

  * **LeetCode 链接**: [70. Climbing Stairs](../../LeetCode%20practice/51-100.ipynb)
  * **AI 关联**: **序列模型与状态转移**。今天的状态只取决于前两天的状态（Markov Property），这是 RNN 和 RL（强化学习）的基础。
  * **核心思路**: $f(n) = f(n-1) + f(n-2)$。这是斐波那契数列。

In [5]:
class Solution:
    def climbStairs(self, n: int) -> int:
        # 边界条件
        if n <= 2: return n
        
        # 动态规划 (滚动数组，省空间)
        # a: n-2, b: n-1
        a, b = 1, 2
        
        for _ in range(3, n + 1):
            # 新的台阶 = 前两阶之和
            temp = a + b
            a = b
            b = temp
            
        return b

### 题目 3: 二叉树的层序遍历 (Binary Tree Level Order Traversal) - Medium (BFS)

  * **LeetCode 链接**: [102. Binary Tree Level Order Traversal](../../LeetCode%20practice/101-150.ipynb)
  * **AI 关联**: **神经网络的 Layer 传播**。输入层 -\> 隐藏层1 -\> 隐藏层2。数据是一层一层往后传的，这本质上就是 BFS（广度优先搜索）。
  * **核心思路**: **队列 (Queue)**。先进先出。

为了方便测试，我写了一个辅助函数把列表 `[3,9,20,null,null,15,7]` 转成树结构。

In [ ]:
from collections import deque
from typing import List, Optional

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

# 辅助函数: 列表转树 (LeetCode 风格)
def build_tree(nodes: List[Optional[int]]) -> Optional[TreeNode]:
    if not nodes: return None
    root = TreeNode(nodes[0])
    queue = deque([root])
    i = 1
    while queue and i < len(nodes):
        node = queue.popleft()
        # 左孩子
        if i < len(nodes) and nodes[i] is not None:
            node.left = TreeNode(nodes[i])
            queue.append(node.left)
        i += 1
        # 右孩子
        if i < len(nodes) and nodes[i] is not None:
            node.right = TreeNode(nodes[i])
            queue.append(node.right)
        i += 1
    return root

class Solution:
    def levelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        if not root: return []
        
        result = []
        queue = deque([root])
        
        while queue:
            level_vals = []
            level_size = len(queue) # 当前这一层有多少个节点
            
            for _ in range(level_size):
                node = queue.popleft()
                level_vals.append(node.val)
                
                if node.left: queue.append(node.left)
                if node.right: queue.append(node.right)
            
            result.append(level_vals)
            
        return result


```python
from collections import deque
from typing import List, Optional

# 定义树节点




# --- 交互式测试 ---
solution = Solution()
print("\n=== 测试二叉树层序遍历 ===")
print("输入格式: 逗号分隔的数字，null用None或n表示")
print("示例: 3,9,20,n,n,15,7 (代表 3是根, 9和20是子节点...)")

raw_input = input("请输入树结构: ").strip()

# 处理输入
try:
    if not raw_input:
        nodes = []
    else:
        # 把 'n' 或 'null' 转成 None，数字转 int
        nodes = []
        for x in raw_input.split(','):
            x = x.strip()
            if x.lower() in ['n', 'null', 'none']:
                nodes.append(None)
            else:
                nodes.append(int(x))
    
    root = build_tree(nodes)
    print(f"层序遍历结果: {solution.levelOrder(root)}")
    
except ValueError:
    print("输入格式错误，请确保输入的是数字！")
```

-----

## 4\. 总结与作业

### 今日复盘

1.  **AI 原理**:
      * **计算图**是有方向的。Forward 顺着走，Backward 逆着走（依赖 Stack 机制）。
      * `grad_fn` 是连接 Tensor 的隐形链条。
2.  **算法技巧**:
      * **Stack**: 处理“对称”、“匹配”、“回溯”类问题。
      * **BFS (Queue)**: 处理“分层”、“最短路径”类问题。

### 课后作业

1.  **运行代码**: 重点体验“有效的括号”和“二叉树”的测试。
2.  **准备模拟面试**:
      * 明天 (Day 6) 我们将进行 **Review & Mock Interview**。
      * 请把 Week 3 的 `src/` 代码再看一遍，明天我会以面试官的身份，让你解释你的 CNN 项目架构。

Day 5 完成后，你的 Stage 1 复盘周已经过半，核心数据结构（数组、链表、哈希、字符串、栈、树）已全部覆盖。

**准备好迎接明天的挑战了吗？**